In [36]:
import torch
import torch.nn as nn
import pickle
from torch.utils.data import DataLoader
import networkx as nx
import numpy as np
from networkx.utils import reverse_cuthill_mckee_ordering
import pandas as pd

In [37]:
# NUMBER_NODES = 10
# NUMBER_NODES = 9
NUMBER_NODES = 7

device = "cuda" if torch.cuda.is_available() else "cpu"

In [38]:
class DNN(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(NUMBER_NODES * NUMBER_NODES, 200)
        self.fc2 = nn.Linear(200, NUMBER_NODES)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [39]:
# model = NeuralNetwork().to(device)
# model.load_state_dict(torch.load("model.pth"))

In [40]:
with open('test_data.pickle', 'rb') as f:
    test_data = pickle.load(f)
test_dataloader = DataLoader(test_data, batch_size=16, shuffle=True)

In [41]:
def getBandWidth(G):
    L = nx.laplacian_matrix(G) # Laplacian Matrix L = Degree matrix - Adjacency matrix
    x, y = np.nonzero(L)
    if not G.edges:
        return 0
    return (x-y).max()

def getCuthillMckeeBand(Graph):
    Graph = nx.Graph(Graph)
    rcm = list(reverse_cuthill_mckee_ordering(Graph))
    L = nx.laplacian_matrix(Graph, nodelist=rcm)
    x, y = np.nonzero(L)
    return (x-y).max()

def get_bandwidth_nn_output(Graph, nodelist):
    print(nodelist)
    Graph = nx.Graph(Graph)
    L = nx.laplacian_matrix(Graph, nodelist=nodelist)
    x, y = np.nonzero(L)
    return (x-y).max()

def getGraph(upperTriangleAdjMatrix):
    dense_adj = np.zeros((NUMBER_NODES, NUMBER_NODES))
    dense_adj = np.zeros((NUMBER_NODES, NUMBER_NODES))
    k = 0
    for i in range(NUMBER_NODES):
        for j in range(NUMBER_NODES):
            if i == j:
                continue
            elif i < j:
                dense_adj[i][j] = upperTriangleAdjMatrix[k]
                k += 1
            else:
                dense_adj[i][j] = dense_adj[j][i]
    return dense_adj


In [42]:
df_compare = pd.DataFrame([], columns=['NN_Output', 'Optimal_band', 'Cuthill_Mckee_band'])

# i = 0
for batch in test_dataloader:
  # if i <= 4:
  #   i += 1
  #   continue
  for x, y in zip(batch[0], batch[1]):
    Graph = getGraph(x)

    print("FELIPE")
    print(Graph)

    # x, y = x.to(device), y
    # pred = model(x)
    # np_true = y.numpy()
    # np_pred = pred.cpu().detach().numpy()
    # # print("==================")
    # # print("True")
    # # print(np_true)
    # # print("Predicted")
    # # print(np_pred)
    # np_pred_rounded = []
    # for i in np_pred:
    #   round = np.round(i)
    #   if round not in np_pred_rounded:
    #     np_pred_rounded.append(round)
    #   else:
    #     floor = np.floor(i)
    #     if floor not in np_pred_rounded:
    #       np_pred_rounded.append(floor)
    #     else:
    #       ceil = np.ceil(i)
    #       np_pred_rounded.append(ceil)
    # print("==================")
    # print("True")
    # print(np_true)
    # print("Predicted")
    # print(np_pred_rounded)
    break
  break
    # output_band = get_bandwidth_nn_output(Graph, np_pred)
    # optimal_band = x[-1]
    # rcm_band = getCuthillMckeeBand(Graph)
    
    # df = df.append({'NN_Output': output_band, 'Optimal_band': optimal_band, 'Cuthill_Mckee_band': rcm_band}, ignore_index=True)

FELIPE
[[0. 0. 0. 0. 1. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 1. 1. 1.]
 [0. 1. 1. 0. 0. 0. 1.]
 [1. 0. 1. 0. 0. 1. 1.]
 [0. 0. 1. 0. 1. 0. 1.]
 [1. 0. 1. 1. 1. 1. 0.]]


In [43]:
# df